In [1]:
import json
import lisp_lib
import collections
import glob


def bracket_conversion(parse_text):
    if "TT_ERR" in parse_text:
        return None
    in_text = False
    new_text = ""
    for i, c in enumerate(parse_text):
        if in_text:
            new_text += c.replace("(", "-LRB-").replace(")", "-RRB-")
            if c == "_" and parse_text[i + 1] == "!":
                in_text = False
        else:
            if c == "_" and parse_text[i + 1] == "!":
                in_text = True
            new_text += c

    return new_text


def read_tag(parse):
    if parse[0] == "span":
        return {"span": tuple(int(x) for x in parse[1:])}
    elif parse[0] == "text":
        return {"text": parse[1:]}
    elif parse[0] == "leaf":
        return {"leaf": int(parse[1])}
    else:
        assert parse[0] == "rel2par"
        return {parse[0]: parse[1]}

def get_file_pair(path, input_file):
    with open(f"{path}{input_file}", "r") as f:
        input_text = f.read()
    with open(f"{path}{input_file}.dis", "r") as f:
        output_text = f.read()
    return input_text, output_text
  
AnnotationPair = collections.namedtuple(
    "AnnotationPair", "identifier input_text main_annotation double_annotation"
)

def build_annotation_pair(files, paths, identifier):
    if identifier.startswith("file"):
        input_file = identifier
    else:
        input_file = f"{identifier}.out"
    main_in, main_out = get_file_pair(paths[TRAIN], input_file)
    double_in, double_out = get_file_pair(paths[DOUBLE], input_file)
    assert main_in == double_in
    if None in [main_out, double_out]:
        return
    return AnnotationPair(identifier, main_in, Parse(main_out), Parse(double_out))

In [2]:
TRAIN, TEST, DOUBLE, SINGLE = "train test double single".split()

DATA_PATH = "./rst_discourse_treebank/"

def build_file_map(data_path=DATA_PATH):
    main_path = f"{data_path}/data/RSTtrees-WSJ-main-1.0/"
    double_path = f"{data_path}/data/RSTtrees-WSJ-double-1.0/"
    paths = {
        TRAIN: f"{main_path}TRAINING/",
        TEST: f"{main_path}TEST/",
        DOUBLE: double_path,
    }
    temp_file_map = collections.defaultdict(list)
    for subset, path in paths.items():
        for filename in glob.glob(f"{path}*.out") + glob.glob(f"{path}file?"):
            identifier = filename.split("/")[-1].split(".")[0]
            temp_file_map[subset].append(identifier)

    files = collections.defaultdict(lambda: collections.defaultdict(list))
    for subset in [TRAIN, TEST]:
        for filename in temp_file_map[subset]:
            if filename in temp_file_map[DOUBLE]:
                files[subset][DOUBLE].append(filename)
            else:
                files[subset][SINGLE].append(filename)

    return paths, files


def is_tag(parse):
    return all(type(x) in [str, float, int] for x in parse)

  
class Subtree(object):
  def __init__(self, parse):
    tags = []
    children = []
    if parse[0] == "Root":
      parse.pop(0)
    self.text = None
    for child in parse:
      if is_tag(child):
        if child[0] == 'text':
          self.text = " ".join(str(i) for i in child[1:])
          for tag_subtree in tags:
            if tag_subtree[0] == 'leaf':
              print(tag_subtree)
              self.leaf = tag_subtree[1]
              print(self.leaf)
          print(self.text)
        tags.append(child)
      else:
        children.append(child)

        
    # Build a right-branching binary tree
    if children:
      self.left_child = Subtree(children[0])
      if len(children) == 2:
        self.right_child = Subtree(children[1])
      else:
        self.right_child = Subtree(children[1:])
    else:
      assert self.text is not None

  def children_in_order(self):
    return self.left_child, self.right_child

    
  
class Parse(object):
    def __init__(self, parse_text):
        maybe_converted = bracket_conversion(parse_text)
        self.is_valid = maybe_converted is not None
        if self.is_valid:
          parse = lisp_lib.parse(maybe_converted)
          self.tree = Subtree(parse)
#           self.edus = get_edu_list(parse)
#           self.span_map = get_span_map(self.tree)


In [3]:
paths, files = build_file_map()

annotation_pairs = [
    build_annotation_pair(files, paths, identifier)
    for identifier in files[TRAIN][DOUBLE]
]

['leaf', 1]
1
_!The Bush administration said_!
['leaf', 2]
2
_!it is submitting a "comprehensive" proposal_!
['leaf', 3]
3
_!for overhauling agricultural trade_!
['leaf', 4]
4
_!that could help break an impasse in the current round of multilateral trade negotiations.<P>_!
['leaf', 5]
5
_!The proposal reiterates the U.S. desire_!
['leaf', 6]
6
_!to scrap or reduce a host of trade-distorting subsidies on farm products._!
['leaf', 7]
7
_!But it would allow considerable flexibility_!
['leaf', 8]
8
_!in determining how and when these goals would be achieved._!
['leaf', 9]
9
_!The U.S. plan also would ease the transition to freer agriculture trade_!
['leaf', 10]
10
_!by allowing some countries to convert non-tariff barriers into tariffs_!
['leaf', 11]
11
_!that, together with existing tariffs, then would be phased out over 10 years.<P>_!
['leaf', 12]
12
_!Trade Representative Carla Hills,_!
['leaf', 13]
13
_!who along with Agriculture Secretary Clayton Yeutter unveiled the proposal,_!
['leaf

['leaf', 1]
1
_!Wang Laboratories Inc. has sold $25 million of assets_!
['leaf', 2]
2
_!and reached agreements in principle_!
['leaf', 3]
3
_!to sell an additional $187 million shortly,_!
['leaf', 4]
4
_!Richard Miller, president, said at the annual meeting.<P>_!
['leaf', 5]
5
_!He said_!
['leaf', 6]
6
_!Wang had reached an agreement with a "major financial firm"_!
['leaf', 7]
7
_!to sell for $150 million its domestic equipment lease portfolio and that of its Wang Credit Corp. subsidiary._!
['leaf', 8]
8
_!He said_!
['leaf', 9]
9
_!it also agreed to sell a portion of its European real estate unit for $37 million.<P>_!
['leaf', 10]
10
_!Mr. Miller said_!
['leaf', 11]
11
_!that Wang has already sold some $12 million of miscellaneous assets_!
['leaf', 12]
12
_!and disclosed_!
['leaf', 13]
13
_!that it had received $13 million from Compaq Computer Corp., Houston, in the previously announced sale of its Stirling, Scotland, plant.<P>_!
['leaf', 14]
14
_!Mr. Miller repeated_!
['leaf', 15]
15


['leaf', 1]
1
_!These are the last words_!
['leaf', 2]
2
_!Abbie Hoffman ever uttered, more or less,_!
['leaf', 3]
3
_!before he killed himself._!
['leaf', 4]
4
_!And You Are There, sort of:<P>_!
['leaf', 5]
5
_!ABBIE:_!
['leaf', 6]
6
_!"I'm OK, Jack._!
['leaf', 7]
7
_!I'm OK."_!
['leaf', 8]
8
_!-LRB-listening-RRB-_!
['leaf', 9]
9
_!"Yeah._!
['leaf', 10]
10
_!I'm out of bed._!
['leaf', 11]
11
_!I got my feet on the floor._!
['leaf', 12]
12
_!Yeah._!
['leaf', 13]
13
_!Two feet._!
['leaf', 14]
14
_!I'll see you Wednesday?_!
['leaf', 15]
15
_!. . . Thursday."<P>_!
['leaf', 16]
16
_!He listens impassively.<P>_!
['leaf', 17]
17
_!ABBIE_!
['leaf', 18]
18
_!-LRB-cont'd.-RRB-:_!
['leaf', 19]
19
_!"I'll always be with you, Jack._!
['leaf', 20]
20
_!Don't worry."<P>_!
['leaf', 21]
21
_!Abbie lies back_!
['leaf', 22]
22
_!and leaves the frame empty.<P>_!
['leaf', 23]
23
_!Of course that wasn't the actual conversation_!
['leaf', 24]
24
_!the late anti-war activist, protest leader and founder of th

In [4]:
from ete3 import Tree, TreeStyle, TextFace
import PyQt5
import rst_lib
from rst_lib import TRAIN, DOUBLE
from ete3 import NodeStyle

ts = TreeStyle()
ts.show_leaf_name = False
ns = NodeStyle()
ns['size'] = 0

In [5]:
def read_in_order(subtree):
  edus = []
  def inorder_helper(tree):
    if tree.text is not None:
      edus.append(tree.text)
    else:
      inorder_helper(tree.left_child)
      inorder_helper(tree.right_child)
  inorder_helper(subtree)
  return edus
      

def get_newick_helper(subtree):
  if subtree.text is not None:
    print(subtree.text, subtree.leaf)
    return str(subtree.leaf)
  else:
    #print(len(subtree.children_in_order()))
    x=  f'({get_newick_helper(subtree.left_child)}, {get_newick_helper(subtree.right_child)})'
    #print(x)
    return x
    #return "(" + ", ".join([get_newick_helper(child) for child in subtree.children_in_order()]) +")"


def get_newick(subtree):
  return get_newick_helper(subtree) + ";"


read_in_order(annotation_pairs[0].main_annotation.tree)

for p in annotation_pairs:
  if p.main_annotation.is_valid and p.double_annotation.is_valid:
    maps = {
      "main": p.main_annotation,
      "double": p.double_annotation
    }
    for which_annotation, annotation in maps.items():
      newick = get_newick(annotation.tree)
      #print(newick)
      face_map = [None] + read_in_order(annotation.tree)
      t = Tree(newick)
      for x in t.traverse():
          x.set_style(ns)
          print(x.name)
          if x.name:
              x.add_face(TextFace("  " +face_map[int(x.name)]), 0)
      t.convert_to_ultrametric()
      t.render(f"binary_parses/{p.identifier}_{which_annotation}.pdf", tree_style=ts, w=700, dpi=300)

_!The Bush administration said_! 1
_!it is submitting a "comprehensive" proposal_! 2
_!for overhauling agricultural trade_! 3
_!that could help break an impasse in the current round of multilateral trade negotiations.<P>_! 4
_!The proposal reiterates the U.S. desire_! 5
_!to scrap or reduce a host of trade-distorting subsidies on farm products._! 6
_!But it would allow considerable flexibility_! 7
_!in determining how and when these goals would be achieved._! 8
_!The U.S. plan also would ease the transition to freer agriculture trade_! 9
_!by allowing some countries to convert non-tariff barriers into tariffs_! 10
_!that, together with existing tariffs, then would be phased out over 10 years.<P>_! 11
_!Trade Representative Carla Hills,_! 12
_!who along with Agriculture Secretary Clayton Yeutter unveiled the proposal,_! 13
_!said_! 14
_!she is confident_! 15
_!it will gain considerable support from the U.S.'s trading partners._! 16
_!Mr. Yeutter,_! 17
_!seeking to allay European objecti

_!The bond market,_! 1
_!which sometimes thrives on bad news,_! 2
_!cheered yesterday's stock market sell-off and perceptions_! 3
_!that the economy is growing weaker.<P>_! 4
_!Early in the day, bonds rose modestly on economists' forecasts_! 5
_!that this week's slate of economic data will portray an economy_! 6
_!headed for trouble._! 7
_!Such news is good for bonds_! 8
_!because economic weakness sometimes causes the Federal Reserve to lower interest rates in an effort_! 9
_!to stimulate the economy_! 10
_!and stave off a recession.<P>_! 11
_!For example, today the Department of Commerce is scheduled to release the September durable goods report._! 12
_!The consensus forecast of 14 economists_! 13
_!surveyed by Dow Jones Capital Markets Report_! 14
_!is for a 1.2% drop in September orders._! 15
_!That would follow a 3.9% advance in August.<P>_! 16
_!Bonds received a bigger boost later in the day_! 17
_!when stock prices moved broadly lower._! 18
_!The Dow Jones Industrial Average fel

_!Japan has climbed up from the ashes of World War II and a gross national product of about $800 per capita_! 1
_!to reach the heavyweight class among industrialized nations._! 2
_!Now this remarkable economic growth seems to be coming to an end_! 3
_!because the government has not converted itself into a modern, democratic, "developed nation" mode of operation.<P>_! 4
_!Until 1980,_! 5
_!when Japan joined the $10,000 per capita GNP club of the advanced countries,_! 6
_!it was a model developing nation._! 7
_!The government built ports, bridges, highways, schools, hospitals and railways._! 8
_!When industries were weak,_! 9
_!it protected them._! 10
_!It gave the Japanese people a value system,_! 11
_!based on the rationalization_! 12
_!that_! 13
_!given the country's lack of natural resources,_! 14
_!they must work hard_! 15
_!to create value through exports_! 16
_!and buy food with the surplus._! 17
_!Individual prosperity inevitably would result.<P>_! 18
_!That system has worked._! 

_!Shearson Lehman Hutton Inc. said_! 1
_!it applied to Taiwanese securities officials for permission_! 2
_!to open brokerage offices in Taipei.<P>_! 3
_!Shearson's application is the first_! 4
_!since the Taiwan Securities and Exchange Commission announced June 21_! 5
_!that it would allow foreign brokerage firms to do business in that country._! 6
_!Taiwan officials are expected to review the Shearson application later this year.<P>_! 7
_!Under current rules, investors in Taiwan can buy overseas stocks only through the purchase of mutual funds_! 8
_!issued by local and foreign investment trusts._! 9
_!The new rules will allow investors to buy foreign stocks directly.<P>_! 10
_!A spokesman for Shearson said_! 11
_!the brokerage service will be directed at individual investors_! 12
_!who want to buy foreign and domestic stocks._! 13
_!"It's an attractive market with good growth opportunities,"_! 14
_!he added._! 15







1



11

14
15
2
3

7

10
12
13
4

8
9
5
6
_!The dollar weakened 

_!Italy's trade deficit narrowed to 2.007 trillion lire_! 1
_!-LRB-$1.49 billion-RRB-_! 2
_!in September from 2.616 trillion lire a year earlier,_! 3
_!the state statistical office Istat said.<P>_! 4
_!The deficit was 466 billion lire in August._! 5
_!For the first nine months, the trade deficit was 14.933 trillion lire,_! 6
_!compared with 10.485 trillion lire in the year-earlier period._! 7
_!Istat said_! 8
_!the statistics are provisional_! 9
_!and aren't seasonally adjusted.<P>_! 10
_!Imports rose 11% to 18.443 trillion lire in September from a year earlier,_! 11
_!while exports rose 17% to 16.436 trillion lire._! 12
_!In the nine months, imports rose 20% to 155.039 trillion lire,_! 13
_!while exports grew 18% to 140.106 trillion lire._! 14
_!Import values are calculated on a cost, insurance and freight_! 15
_!-LRB-c.i.f.-RRB-_! 16
_!basis,_! 17
_!while exports are accounted for on a free-on-board_! 18
_!-LRB-f.o.b.-RRB-_! 19
_!basis._! 20









8






4
5

9
10
11
12
13
14

17

...

_!The West German retailer ASKO Deutsche Kaufhaus AG plans to challenge the legality of a widely employed anti-takeover defense of companies in the Netherlands.<P>_! 1
_!The eventual court decision could become a landmark in Dutch corporate law_! 2
_!because the lawsuit_! 3
_!ASKO plans to file_! 4
_!would be the first_! 5
_!to challenge the entire principle and practice of companies_! 6
_!issuing voting preferred shares to management-controlled trusts_! 7
_!to dilute voting power of common stockholders.<P>_! 8
_!Up to now only specific aspects of these defenses have been challenged, though unsuccessfully,_! 9
_!ASKO's Dutch lawyers noted.<P>_! 10
_!Should the courts uphold the validity of this type of defense,_! 11
_!ASKO will then ask the court to overturn such a vote-diluting maneuver_! 12
_!recently deployed by Koninklijke Ahold NV._! 13
_!ASKO says_! 14
_!the Dutch-based international food retailer hadn't reasonable grounds_! 15
_!to issue preferred stock to a friendly trust_! 16


_!Norfolk Southern Corp. directors authorized the railroad company to buy back as many as 45 million of its shares,_! 1
_!which would have a current value of more than $1.7 billion.<P>_! 2
_!The buy-back,_! 3
_!coupled with a nearly completed earlier purchase of 20 million shares,_! 4
_!would reduce shares outstanding by more than 26%._! 5
_!The Norfolk, Va., company has 172.2 million shares outstanding.<P>_! 6
_!In a statement, Arnold B. McKinnon, chairman and chief executive officer, noted_! 7
_!that the new repurchase program "should serve to enhance shareholder value."_! 8
_!A spokeswoman said_! 9
_!the company will finance the buy-back with cash on hand, borrowing and "cash_! 10
_!Norfolk expects to generate."<P>_! 11
_!Analysts said_! 12
_!they expected the action,_! 13
_!and investors applauded the move._! 14
_!In composite trading on the New York Stock Exchange, Norfolk Southern shares closed at $37.875, up $1.125.<P>_! 15
_!Still, analysts don't expect the buy-back to signific

_!INSURERS ARE FACING billions of dollars in damage claims from the California quake._! 1
_!But most businesses in the Bay area,_! 2
_!including Silicon Valley,_! 3
_!weren't greatly affected._! 4
_!Computer and software companies in the region are expecting virtually no long-term disruption in shipments._! 5
_!Also, investors quickly singled out stocks of companies_! 6
_!expected to profit or suffer from the disaster.<P>_! 7
_!Leveraged buy-outs may be curbed by two rules in pending congressional legislation._! 8
_!The provisions, in deficit-reduction bills_! 9
_!recently passed by the House and Senate,_! 10
_!could raise the price tags of such deals by up to 10%_! 11
_!and cool the takeover boom.<P>_! 12
_!A bill_! 13
_!giving the Transportation Department the power_! 14
_!to block airline leveraged buy-outs_! 15
_!cleared a House panel._! 16
_!But Secretary Skinner said_! 17
_!he would urge Bush to veto the bill.<P>_! 18
_!Housing starts sank 5.2% in September to a seven-year low._!

_!CALIFORNIA STRUGGLED with the aftermath of a Bay area earthquake.<P>_! 1
_!As aftershocks shook the San Francisco Bay area,_! 2
_!rescuers searched through rubble for survivors of Tuesday's temblor,_! 3
_!and residents picked their way through glass-strewn streets._! 4
_!In Oakland, hopes faded_! 5
_!for finding any more survivors within the concrete and steel from the collapse of an interstate highway._! 6
_!At least 270 people were reported killed and 1,400 injured in the rush-hour tremor_! 7
_!that caused billions of dollars of damage along 100 miles of the San Andreas fault._! 8
_!Bush declared the region a major disaster area_! 9
_!and the military was mobilized_! 10
_!to prevent looting.<P>_! 11
_!The baseball commissioner said_! 12
_!the third game of the World Series between the Giants and the Athletics would be played Tuesday in Candlestick Park.<P>_! 13
_!HONECKER WAS OUSTED as leader of East Germany amid growing unrest.<P>_! 14
_!The 77-year-old official,_! 15
_!who oversa

_!The government is sharpening its newest weapon against white-collar defendants :_! 1
_!the power_! 2
_!to prevent them from paying their legal bills._! 3
_!And defense lawyers are warning_! 4
_!that they won't stick around_! 5
_!if they don't get paid._! 6
_!The issue has come to a boil in Newark, N.J.,_! 7
_!where federal prosecutors have warned lawyers for Eddie Antar_! 8
_!that_! 9
_!if the founder and former chairman of Crazy Eddie Inc. is indicted,_! 10
_!the government may move to seize the money_! 11
_!that Mr. Antar is using to pay legal fees._! 12
_!The warning by the U.S. attorney's office follows two decisions by the U.S. Supreme Court last June._! 13
_!In those cases, the high court ruled_! 14
_!that federal law gives prosecutors broad authority_! 15
_!to seize assets of people_! 16
_!accused of racketeering and drug-related crimes,_! 17
_!including fees_! 18
_!paid to lawyers before an indictment._! 19
_!If the government succeeds in seizing Mr. Antar's assets,_! 20
_!he